In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
import art
import asyncio
from dotenv import load_dotenv
import json
import openai
import random
import re
from typing import TypedDict

load_dotenv()


class TemporalCluePuzzle(TypedDict):
    num_clues: int
    prompt: str
    solution: dict[str, str]


puzzles: list[TemporalCluePuzzle] = json.load(open("./data/temporal-clue/puzzles.json"))
val_puzzles = puzzles[:64]
test_puzzles = puzzles[64:128]
train_puzzles = puzzles[128:]
random.seed(42)
random.shuffle(train_puzzles)


api = art.LocalAPI(wandb_project="agent-reinforcement-training")
model = await api.get_or_create_model(
    name="temporal-clue-002", base_model="Qwen/Qwen2.5-14B-Instruct"
)


async def rollout(
    client: openai.AsyncOpenAI, puzzle: TemporalCluePuzzle
) -> art.Trajectory:
    messages: art.Messages = [{"role": "user", "content": puzzle["prompt"]}]
    chat_completion = await client.chat.completions.create(
        messages=messages, model=model.name
    )
    choice = chat_completion.choices[0]
    content = choice.message.content
    assert isinstance(content, str)
    num_correct = 0
    for key, value in puzzle["solution"].items():
        if matches := re.findall(rf"{key}\. ([A-Za-z \.:-]+)", content):
            match = matches[-1]
            if match.strip().lower() == value.lower():
                num_correct += 1
    reward = acc = num_correct / len(puzzle["solution"])
    return art.Trajectory(
        messages_and_choices=[*messages, choice], reward=reward, metrics={"acc": acc}
    )


stride = 32
for i in range(await model.get_iteration(), 1_000):
    async with model.openai_client(
        estimated_token_usage=850, verbosity=2
    ) as openai_client:
        val_groups, train_groups = await asyncio.gather(
            art.gather_groups(
                (
                    (rollout(openai_client, puzzle) for _ in range(2))
                    for puzzle in val_puzzles
                ),
                pbar_desc="val",
            ),
            art.gather_groups(
                (
                    (rollout(openai_client, puzzle) for _ in range(50))
                    for puzzle in train_puzzles[i * stride : (i + 1) * stride]
                ),
                pbar_desc="train",
            ),
        )
    _, _ = await asyncio.gather(
        model.save(val_groups),
        model.tune(train_groups, config=art.TuneConfig(plot_tensors=True, verbosity=2)),
    )

$ vllm serve Qwen/Qwen2.5-14B-Instruct --block-size=32 --disable-log-requests --enable-chunked-prefill --enable-prefix-caching --enforce-eager --gpu-memory-utilization=0.95 --max-num-seqs=4096 --max-num-batched-tokens=16384 --num-scheduler-steps=16 --preemption-mode=swap --return-tokens-as-token-ids --swap-space=80 --tensor-parallel-size=2 --tool-call-parser=hermes --served-model-name=temporal-clue-002 --port=8000 --api-key=default
INFO 03-14 01:55:08 __init__.py:207] Automatically detected platform cuda.
INFO 03-14 01:55:08 api_server.py:912] vLLM API server version 0.7.3
INFO 03-14 01:55:08 api_server.py:913] args: Namespace(subparser='serve', model_tag='Qwen/Qwen2.5-14B-Instruct', config='', host=None, port=8000, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key='default', lora_modules=None, prompt_adapters=None, chat_template=None, chat_template_content_format='auto', response_role='assistant', ssl_keyfil

Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  12% Completed | 1/8 [00:00<00:04,  1.73it/s]
Loading safetensors checkpoint shards:  25% Completed | 2/8 [00:01<00:03,  1.69it/s]
Loading safetensors checkpoint shards:  38% Completed | 3/8 [00:01<00:02,  1.69it/s]
Loading safetensors checkpoint shards:  50% Completed | 4/8 [00:01<00:01,  2.32it/s]
Loading safetensors checkpoint shards:  62% Completed | 5/8 [00:02<00:01,  2.07it/s]
Loading safetensors checkpoint shards:  75% Completed | 6/8 [00:03<00:00,  2.08it/s]
Loading safetensors checkpoint shards:  88% Completed | 7/8 [00:03<00:00,  1.90it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:04<00:00,  1.82it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:04<00:00,  1.89it/s]



INFO 03-14 01:55:27 model_runner.py:1115] Loading model weights took 13.9281 GB
(VllmWorkerProcess pid=67380) INFO 03-14 01:55:28 model_runner.py:1115] Loading model weights took 13.9281 GB
(VllmWorkerProcess pid=67380) INFO 03-14 01:55:30 worker.py:267] Memory profiling takes 2.75 seconds
(VllmWorkerProcess pid=67380) INFO 03-14 01:55:30 worker.py:267] the current vLLM instance can use total_gpu_memory (79.11GiB) x gpu_memory_utilization (0.95) = 75.15GiB
(VllmWorkerProcess pid=67380) INFO 03-14 01:55:30 worker.py:267] model weights take 13.93GiB; non_torch_memory takes 1.57GiB; PyTorch activation peak memory takes 1.45GiB; the rest of the memory reserved for KV Cache is 58.21GiB.
INFO 03-14 01:55:31 worker.py:267] Memory profiling takes 3.03 seconds
INFO 03-14 01:55:31 worker.py:267] the current vLLM instance can use total_gpu_memory (79.11GiB) x gpu_memory_utilization (0.95) = 75.15GiB
INFO 03-14 01:55:31 worker.py:267] model weights take 13.93GiB; non_torch_memory takes 1.82GiB; Py

INFO:     Started server process [66959]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


INFO 03-14 01:56:21 chat_utils.py:332] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
INFO:     127.0.0.1:54644 - "POST /v1/chat/completions HTTP/1.1" 200 OK
vLLM server started succesfully. Logs can be found at ./logs/vllm.log


val:   0%|          | 0/128 [00:00<?, ?it/s]

train:   0%|          | 0/1600 [00:00<?, ?it/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bradhilton to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
